In [1]:
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from transformers import AutoTokenizer

from captum.attr import visualization
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
from transformers import BertTokenizer, BertModel

/data/AI/backup/namlt/namlt/temp/anaconda3/envs/bert/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
model = BertForSequenceClassification.from_pretrained("jackaduma/SecBERT", num_labels=1).to("cuda")
model.eval()
tokenizer = AutoTokenizer.from_pretrained('jackaduma/SecBERT')
# initialize the explanations generator



explanations = Generator(model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at jackaduma/SecBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
classifications = ["Normal", "Attack"]

In [2]:
checkpoint = torch.load('/data/AI/backup/namlt/namlt/Hidden/Transformer-Explainability/sqli_model.pt')

In [3]:
dict = checkpoint['state_dict']
dict1 = dict.copy()
for name in dict.keys():
    name_new = name.replace('bert_model', 'bert')
    dict1[name_new] = dict[name]
    del dict1[name]

In [6]:
dict2 = model.state_dict()
dict2["bert.embeddings.position_ids"]

tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
          42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
          56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
          70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
          84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
          98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
         112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
         126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
         140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
         154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
         168, 169, 170, 171, 172, 173, 174, 175, 176

In [7]:
dict1["bert.embeddings.position_embeddings.weight"] = dict["bert_model.embeddings.position_embeddings.weight"]
dict1["bert.embeddings.word_embeddings.weight"] = dict["bert_model.embeddings.word_embeddings.weight"]
dict1["classifier.weight"] = dict["linear.weight"]
dict1["classifier.bias"] = dict["linear.bias"]
dict1["bert.embeddings.position_ids"] = dict2["bert.embeddings.position_ids"]

In [10]:
model.load_state_dict(dict1)

<All keys matched successfully>

In [48]:
text_batch = ["http://newspaper.com/items.php?id=2 and 1=2"]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")
token_type_ids = encoding['token_type_ids'].to("cuda")

# true class is positive - 1
true_class = 0

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = model(input_ids=input_ids, attention_mask=attention_mask)
# print(output)
output1 = output.logits
classification = round(torch.sigmoid(output1.clone().detach()).cpu().numpy()[0][0])
# get class name
class_name = classifications[classification]
print(class_name)
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "Attack":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
# print(output)
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][0],
                                classification,
                                true_class,
                                true_class,
                                1,
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)


Attack
[('[CLS]', -0.0), ('http', -1.0), (':', -0.2826794385910034), ('/', -0.310763418674469), ('/', -0.2647920250892639), ('newspaper', -0.2993714213371277), ('.', -0.035090893507003784), ('com', -0.08438950777053833), ('/', -0.01811852492392063), ('items', -0.0), ('.', -0.0491681806743145), ('php', -0.06259629130363464), ('?', -0.08576725423336029), ('id', -0.09930574148893356), ('=', -0.20088057219982147), ('2', -0.16546565294265747), ('and', -0.10193020850419998), ('1', -0.21137672662734985), ('=', -0.4089820384979248), ('2', -0.2865893542766571), ('[SEP]', -0.15829527378082275)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,1 (1.00),0,1.00,[CLS] http : / / newspaper . com / items . php ? id = 2 and 1 = 2 [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,1 (1.00),0,1.00,[CLS] http : / / newspaper . com / items . php ? id = 2 and 1 = 2 [SEP]


In [29]:
text_batch = ["select * from users where id  =  1 or 1#\"  \(   union select 1,version  \(    \)   -- 1"]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")
token_type_ids = encoding['token_type_ids'].to("cuda")

# true class is positive - 1
true_class = 0

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = model(input_ids=input_ids, attention_mask=attention_mask).logits
classification = round(torch.sigmoid(output.clone().detach()).cpu().numpy()[0][0])
print(classification)

1


/data/AI/backup/namlt/namlt/temp/anaconda3/envs/bert/lib/python3.10/site-packages/transformers/modeling_utils.py:962: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
